Author:
        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

# Airline Passenger Satisfaction Classification

https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction/data


<p>
    <img src="https://www.travelandleisure.com/thmb/h97kSvljd2QYH2nUy3Y9ZNgO_pw=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/plane-data-BUSYROUTES1217-f4f84b08d47f4951b11c148cee2c3dea.jpg" width=600>
</p>

## 1. Load Libraries

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import torch
import pandas as pd

## 2. Load  Dataset

In [3]:
df_train = pd.read_csv(path_append + '../data/Airline Customer Satisfaction/train.csv')
df_test = pd.read_csv(path_append + '../data/Airline Customer Satisfaction/test.csv')
df_train.head()
df_test.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,...,2,2,2,2,4,2,4,0,20.0,satisfied


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [5]:
# Check labels count
df_train[['satisfaction']].value_counts()

satisfaction           
neutral or dissatisfied    58879
satisfied                  45025
Name: count, dtype: int64

## 3. Preprocessing

In [6]:
from tools.preprocessing.data_frame import process_dataframe

target_columns = ['satisfaction']
drop_columns = ['Unnamed: 0','id']

# Assuming df_train and df_test are your initial dataframes
df_train_length = len(df_train)
df_test_length = len(df_test)

# Concatenate the dataframes
df = pd.concat([df_train, df_test], axis=0)

# Process the combined dataframe
df, description_train = process_dataframe(df, target_columns, drop_columns=drop_columns)

# Split the dataframe back into training and test sets
df_train = df.iloc[:df_train_length]
df_test = df.iloc[df_train_length:]

num_features = description_train['num_features']
num_classes = description_train['num_classes']

print(f"Number of features after scaling: {num_features}")
print(f"Number of classes after scaling: {num_classes}")
description_train

Column 'Gender' has 2 unique values.
Column 'Customer Type' has 2 unique values.
Column 'Type of Travel' has 2 unique values.
Column 'Class' has 3 unique values.
Column 'satisfaction' has 2 unique values.
Number of features after scaling: 24
Number of classes after scaling: 1


{'num_features': 24,
 'num_classes': 1,
 'encoded_columns': Index(['Gender', 'Customer Type', 'Type of Travel', 'Class'], dtype='object'),
 'target_encoded_columns': Index(['satisfaction'], dtype='object'),
 'scalers': {'Age': 'minmax',
  'Arrival Delay in Minutes': 'robust',
  'Baggage handling': 'standard',
  'Checkin service': 'robust',
  'Cleanliness': 'minmax',
  'Departure Delay in Minutes': 'robust',
  'Departure/Arrival time convenient': 'minmax',
  'Ease of Online booking': 'minmax',
  'Flight Distance': 'robust',
  'Food and drink': 'minmax',
  'Gate location': 'minmax',
  'Inflight entertainment': 'minmax',
  'Inflight service': 'standard',
  'Inflight wifi service': 'minmax',
  'Leg room service': 'minmax',
  'On-board service': 'minmax',
  'Online boarding': 'minmax',
  'Seat comfort': 'minmax'},
 'target_scalers': {}}

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             103904 non-null  float64
 1   Customer Type                      103904 non-null  float64
 2   Age                                103904 non-null  float64
 3   Type of Travel                     103904 non-null  float64
 4   Flight Distance                    103904 non-null  float64
 5   Inflight wifi service              103904 non-null  float64
 6   Departure/Arrival time convenient  103904 non-null  float64
 7   Ease of Online booking             103904 non-null  float64
 8   Gate location                      103904 non-null  float64
 9   Food and drink                     103904 non-null  float64
 10  Online boarding                    103904 non-null  float64
 11  Seat comfort                       1039

In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25583 entries, 103904 to 129486
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             25583 non-null  float64
 1   Customer Type                      25583 non-null  float64
 2   Age                                25583 non-null  float64
 3   Type of Travel                     25583 non-null  float64
 4   Flight Distance                    25583 non-null  float64
 5   Inflight wifi service              25583 non-null  float64
 6   Departure/Arrival time convenient  25583 non-null  float64
 7   Ease of Online booking             25583 non-null  float64
 8   Gate location                      25583 non-null  float64
 9   Food and drink                     25583 non-null  float64
 10  Online boarding                    25583 non-null  float64
 11  Seat comfort                       25583 non-nul

In [9]:
# Custom Dataset Class
class AirlineDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        vals = torch.tensor(self.x[index], dtype = torch.float32)
        label = torch.tensor(self.y[index], dtype= torch.long)
        return vals, label

In [10]:

# Split the data into train, validation and test sets
X_train, y_train = df_train.iloc[:, :-1].values, df_train.iloc[:, -1:].values
X_test, y_test = df_test.iloc[:, :-1].values, df_test.iloc[:,-1:].values

trainset = AirlineDataset(X_train, y_train)
testset = AirlineDataset(X_test, y_test)

print(f"Labeled Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"Labeled Testset Shape: {len(testset)}, {testset.x.shape[1]}")

Labeled Trainset Shape: 103904, 24
Labeled Testset Shape: 25583, 24


In [11]:
num_features =  trainset.x.shape[1]
num_classes =  trainset.y.shape[1]

num_features, num_classes 

(24, 1)

# Training

In [12]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

# Set the data configuration
data_config = DataConfig(dataset_name = 'airline_satisfaction', task_type='binary_classification', obs_shape=[num_features], label_size=num_classes)

#  Set the ML parameters
ml_params = MLParameters(ccnet_network = 'tabnet', encoder_network = 'none')
ml_params.algorithm.error_function = 'mae'
ml_params.model.ccnet_config.num_layers = 3

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False)

In [13]:
trainer_hub.train(trainset, testset)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junhopark (ccnets). Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (.\..\saved\airline_satisfaction\causal-learning)... Done. 0.0s


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/1623 [00:00<?, ?it/s]

[0/100][100/1623][Time 10.21]
Unified LR across all optimizers: 0.00019907191565870155
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.2052	Gen: 0.3902	Rec: 0.3924	E: 0.2030	R: 0.2075	P: 0.5774
--------------------Test Metrics------------------------
accuracy: 0.8203
precision: 0.8601
recall: 0.8255
f1_score: 0.8425

[0/100][200/1623][Time 9.08]
Unified LR across all optimizers: 0.00019815726328921765
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.1004	Gen: 0.2411	Rec: 0.2402	E: 0.1012	R: 0.0995	P: 0.3810
--------------------Test Metrics------------------------
accuracy: 0.8750
precision: 0.8889
recall: 0.8889
f1_score: 0.8889

[0/100][300/1623][Time 9.04]
Unified LR across all optimizers: 0.00019724681336564005
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0809	Gen: 0.2044	Rec: 0.2031	E: 0.0822	R: 0.0796	P: 0.3266
--------------------Test Metrics------------------------
ac

...

[58/100][66/1623][Time 9.53]
Unified LR across all optimizers: 2.6122214762475417e-06
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0066	Gen: 0.0340	Rec: 0.0338	E: 0.0067	R: 0.0065	P: 0.0612
--------------------Test Metrics------------------------
accuracy: 0.9609
precision: 0.9927
recall: 0.9379
f1_score: 0.9645

[58/100][166/1623][Time 9.30]
Unified LR across all optimizers: 2.6002194087788034e-06
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0073	Gen: 0.0348	Rec: 0.0346	E: 0.0075	R: 0.0070	P: 0.0621
--------------------Test Metrics------------------------
accuracy: 0.9844
precision: 0.9724
recall: 1.0000
f1_score: 0.9860

[58/100][266/1623][Time 9.23]
Unified LR across all optimizers: 2.5882724858010068e-06
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0066	Gen: 0.0347	Rec: 0.0344	E: 0.0068	R: 0.0063	P: 0.0625
--------------------Test Metrics------------------------
a

Iterations:   0%|          | 0/1623 [00:00<?, ?it/s]

[59/100][43/1623][Time 8.79]
Unified LR across all optimizers: 2.4266659462586118e-06
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0071	Gen: 0.0338	Rec: 0.0335	E: 0.0074	R: 0.0067	P: 0.0602
--------------------Test Metrics------------------------
accuracy: 0.9570
precision: 0.9434
recall: 0.9868
f1_score: 0.9646

[59/100][143/1623][Time 8.77]
Unified LR across all optimizers: 2.4155164290082876e-06
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0073	Gen: 0.0347	Rec: 0.0345	E: 0.0074	R: 0.0071	P: 0.0619
--------------------Test Metrics------------------------
accuracy: 0.9453
precision: 0.9343
recall: 0.9624
f1_score: 0.9481

[59/100][243/1623][Time 8.65]
Unified LR across all optimizers: 2.4044181391364643e-06
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0072	Gen: 0.0344	Rec: 0.0341	E: 0.0075	R: 0.0069	P: 0.0612
--------------------Test Metrics------------------------
a

Iterations:   0%|          | 0/1623 [00:00<?, ?it/s]

[60/100][20/1623][Time 8.94]
Unified LR across all optimizers: 2.25429109601775e-06
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0070	Gen: 0.0346	Rec: 0.0343	E: 0.0072	R: 0.0067	P: 0.0620
--------------------Test Metrics------------------------
accuracy: 0.9727
precision: 0.9856
recall: 0.9648
f1_score: 0.9751

[60/100][120/1623][Time 8.82]
Unified LR across all optimizers: 2.243933569263377e-06
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0068	Gen: 0.0348	Rec: 0.0347	E: 0.0070	R: 0.0067	P: 0.0627
--------------------Test Metrics------------------------
accuracy: 0.9688
precision: 0.9714
recall: 0.9714
f1_score: 0.9714

[60/100][220/1623][Time 8.90]
Unified LR across all optimizers: 2.2336236310217107e-06
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0065	Gen: 0.0345	Rec: 0.0343	E: 0.0067	R: 0.0064	P: 0.0623
--------------------Test Metrics------------------------
accu

In [ ]:
trainer_hub.test(testset)